<a href="https://colab.research.google.com/github/SandhyaHV-7/Harnessing-Machine-Learning-for-Identification-of-Medicinal-Plants/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nltk tensorflow transformers


In [2]:
import nltk
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np
import random
import json
import pickle


nltk.download('punkt_tab')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
{
    "intents": [
        {
            "intent": "greeting",
            "patterns": ["Hi", "Hello", "How are you?", "Hey", "Good morning"],
            "responses": ["Hello!", "Hi there!", "Greetings!", "How can I help you?"]
        },
        {
            "intent": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later", "Take care"],
            "responses": ["Goodbye!", "See you later!", "Take care!"]
        },
        {
            "intent": "thanks",
            "patterns": ["Thanks", "Thank you", "Thanks a lot"],
            "responses": ["You're welcome!", "No problem!", "Glad I could help!"]
        },
        {
            "intent": "inform",
            "patterns": ["Tell me a joke", "Tell me something interesting", "What is the weather like?"],
            "responses": ["Why don't skeletons fight each other? They don't have the guts!", "It's always sunny in here!", "I don't know, but I think it's cloudy."]
        }
    ]
}


{'intents': [{'intent': 'greeting',
   'patterns': ['Hi', 'Hello', 'How are you?', 'Hey', 'Good morning'],
   'responses': ['Hello!', 'Hi there!', 'Greetings!', 'How can I help you?']},
  {'intent': 'goodbye',
   'patterns': ['Bye', 'Goodbye', 'See you later', 'Take care'],
   'responses': ['Goodbye!', 'See you later!', 'Take care!']},
  {'intent': 'thanks',
   'patterns': ['Thanks', 'Thank you', 'Thanks a lot'],
   'responses': ["You're welcome!", 'No problem!', 'Glad I could help!']},
  {'intent': 'inform',
   'patterns': ['Tell me a joke',
    'Tell me something interesting',
    'What is the weather like?'],
   'responses': ["Why don't skeletons fight each other? They don't have the guts!",
    "It's always sunny in here!",
    "I don't know, but I think it's cloudy."]}]}

In [4]:

lemmatizer = WordNetLemmatizer()
intents = json.loads('''{ "intents": [ { "intent": "greeting", "patterns": ["Hi", "Hello", "How are you?", "Hey", "Good morning"], "responses": ["Hello!", "Hi there!", "Greetings!", "How can I help you?"] }, { "intent": "goodbye", "patterns": ["Bye", "Goodbye", "See you later", "Take care"], "responses": ["Goodbye!", "See you later!", "Take care!"] }, { "intent": "thanks", "patterns": ["Thanks", "Thank you", "Thanks a lot"], "responses": ["You're welcome!", "No problem!", "Glad I could help!"] }, { "intent": "inform", "patterns": ["Tell me a joke", "Tell me something interesting", "What is the weather like?"], "responses": ["Why don't skeletons fight each other? They don't have the guts!", "It's always sunny in here!", "I don't know, but I think it's cloudy."] } ] } ''')

patterns = []
responses = []
labels = []


for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        patterns.extend(word_list)
        responses.extend(intent['responses'])
        labels.append(intent['intent'])


patterns = [lemmatizer.lemmatize(w.lower()) for w in patterns]
patterns = sorted(list(set(patterns)))


label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [6]:

training_sentences = []
training_labels = []


for intent in intents['intents']:
    for pattern in intent['patterns']:

        word_list = nltk.word_tokenize(pattern)
        training_sentences.append(' '.join(word_list))
        training_labels.append(intent['intent'])

def bow(sentence, words):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(w.lower()) for w in sentence_words]
    bag = [0]*len(words)
    for s in sentence_words:
        for i, word in enumerate(words):
            if word == s:
                bag[i] = 1
    return(np.array(bag))


training_sentences_bow = [bow(sentence, patterns) for sentence in training_sentences]
training_labels_encoded =  label_encoder.transform(training_labels)
from tensorflow.keras.utils import to_categorical

training_labels_encoded =  to_categorical(training_labels_encoded, num_classes=len(set(labels)))

pickle.dump(patterns, open("patterns.pkl", "wb"))
pickle.dump(label_encoder, open("label_encoder.pkl", "wb"))


In [7]:

model = Sequential()
model.add(Dense(128, input_shape=(len(patterns),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(set(labels)), activation='softmax'))



model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


from tensorflow.keras.utils import to_categorical
training_labels_encoded = to_categorical(labels, num_classes=len(set(labels)))


model.fit(np.array(training_sentences_bow), training_labels_encoded, epochs=200, batch_size=5, verbose=1)


model.save("chatbot_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4250 - loss: 1.3506
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4917 - loss: 1.2827 
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3667 - loss: 1.3294 
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6250 - loss: 1.2619 
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5583 - loss: 1.2448 
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6083 - loss: 1.2345 
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5167 - loss: 1.1942 
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6333 - loss: 1.2435 
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6000 - loss: 1.2018 
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6333 - loss: 1.1634 
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9000 - loss: 1.1198 
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8000 - loss

In [ ]:
def chatbot_response(text):

    model = tf.keras.models.load_model('chatbot_model.h5')
    patterns = pickle.load(open("patterns.pkl", "rb"))
    label_encoder = pickle.load(open("label_encoder.pkl", "rb"))


    bow_input = bow(text, patterns)
    prediction = model.predict(np.array([bow_input]))[0]
    predicted_class = np.argmax(prediction)


    intent = label_encoder.inverse_transform([predicted_class])


    for i in intents['intents']:
        if i['intent'] == intent:
            response = random.choice(i['responses'])
            return response


while True:
    message = input("You: ")
    if message.lower() in ["exit", "quit", "bye"]:
        print("Bot: Goodbye!")
        break
    response = chatbot_response(message)
    print(f"Bot: {response}")


You: Hi Preksha


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Bot: Hi there!
You: how are you?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Bot: Greetings!
